In [118]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model, model_selection, metrics, svm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [47]:
df = pd.read_excel('data/DataSet_w_NA.xlsx', sheet_name="Испорченные факты")
df = df.dropna()
df =pd.pivot_table(df, values=['Продажи, руб', 'Продажи, шт','Повторение заказа','Маржинальная прибыль','Повторение товара'], index=["Факты.Товар ID"],
                     aggfunc={'Продажи, шт': [np.median, np.sum],
                              'Продажи, руб': np.sum,
                              'Повторение заказа': np.sum,
                              'Маржинальная прибыль': np.sum
                        
                               })
newname=df.columns.map('_'.join)
df.columns=newname
df=df.reset_index()
total_sale=df['Продажи, руб_sum'].sum()
df['Доля']=df['Продажи, руб_sum']/total_sale * 100
df['Доля'].sum()
df = df.sort_values(by=('Продажи, руб_sum'), ascending=False)
df=df.assign(sum_d=df['Доля'].cumsum())
df.loc[(df['sum_d'] <= 80), 'ABC'] = 'A'
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'ABC'] = 'B'
df.loc[(df['sum_d'] > 95), 'ABC'] = 'C'

df['Стоимость, руб'] = df['Продажи, руб_sum']/df['Продажи, шт_sum']
df['Продажи в следующем периоде']= (df['Продажи, шт_sum'] + df['Продажи, шт_median']) *df['Стоимость, руб']
total_sale_next=df['Продажи в следующем периоде'].sum()
df['Доля_будущая']=df['Продажи в следующем периоде']/total_sale_next * 100
df = df.sort_values(by=('Продажи в следующем периоде'), ascending=False)
df=df.assign(sum_d_next=df['Доля_будущая'].cumsum())
df['Доля_будущая'].sum()
df.loc[(df['sum_d_next'] <= 80), 'ABC_next'] = 'A'
df.loc[(df['sum_d_next'] > 80) & (df['sum_d_next'] <= 95), 'ABC_next'] = 'B'
df.loc[(df['sum_d_next'] > 95), 'ABC_next'] = 'C'
df.loc[(df['ABC'] != df['ABC_next']), 'Изменение класса'] = 1
df.loc[(df['ABC'] == df['ABC_next']), 'Изменение класса'] = 0
df['Изменение класса'].sum()
df.loc[(df['sum_d'] <= 80), 'class'] = '0'
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'class'] = '1'
df.loc[(df['sum_d'] > 95), 'class'] = '2'
df.loc[(df['sum_d_next'] <= 80), 'class_next'] = '0'
df.loc[(df['sum_d_next'] > 80) & (df['sum_d_next'] <= 95), 'class_next'] = '1'
df.loc[(df['sum_d_next'] > 95), 'class_next'] = '2'
df

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC,"Стоимость, руб",Продажи в следующем периоде,Доля_будущая,sum_d_next,ABC_next,Изменение класса,class,class_next
1388,PROD0001406,1.259996e+07,17,30799912.0,4.0,20.0,2.681845,2.681845,A,1.539996e+06,3.695989e+07,2.691941,2.691941,A,0.0,0,0
1411,PROD0001430,-9.055392e+05,7,11319240.0,6.0,6.0,0.985602,4.862673,A,1.886540e+06,2.263848e+07,1.648854,4.340795,A,0.0,0,0
450,PROD0000454,3.876520e+06,23,13726692.0,3.0,31.0,1.195226,3.877071,A,4.427965e+05,1.505508e+07,1.096523,5.437318,A,0.0,0,0
483,PROD0000488,1.746230e-10,15,10935288.0,5.0,39.0,0.952170,5.814843,A,2.803920e+05,1.233725e+07,0.898572,6.335891,A,0.0,0,0
1629,PROD0001657,2.047488e+06,6,9187447.5,4.0,12.0,0.799980,9.126346,A,7.656206e+05,1.224993e+07,0.892213,7.228103,A,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,PROD0001257,1.050000e+03,12,4200.0,2.5,5.0,0.000366,99.997704,C,8.400000e+02,6.300000e+03,0.000459,99.998575,C,0.0,2,2
1108,PROD0001119,2.220000e+02,13,4884.0,1.5,6.0,0.000425,99.996103,C,8.140000e+02,6.105000e+03,0.000445,99.999020,C,0.0,2,2
801,PROD0000808,1.702000e+03,2,4048.0,2.5,5.0,0.000352,99.998056,C,8.096000e+02,6.072000e+03,0.000442,99.999462,C,0.0,2,2
1675,PROD0001703,1.411200e+03,1,2880.0,2.0,2.0,0.000251,99.999929,C,1.440000e+03,5.760000e+03,0.000420,99.999882,C,0.0,2,2


In [48]:
allinput = df[ ['Продажи, руб_sum', 'Доля'] ] #фичи
alloutput = df["ABC"]

In [120]:
#обертки
def predict_Values(method_Obj, X_train, Y_train):
    method_Obj.fit(X_train, Y_train)
    Y_pred=method_Obj.predict(X_test)
    return Y_pred

def print_Report(Y_test, Y_pred, method, names):
    print(method)
    print(classification_report(Y_test, Y_pred, target_names=names))

In [121]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(allinput, alloutput, test_size=0.2)
name_values = ['A', 'B', 'C']

In [129]:
rfc_Y = predict_Values(RandomForestClassifier(), X_train, Y_train)
print_Report(Y_test, rfc_Y, 'Метод RandomForestClassifier', name_values)

SVC_Y = predict_Values(svm.SVC(), X_train, Y_train)
print_Report(Y_test, SVC_Y, 'Метод Support Vector Machines', name_values)

knc_Y = predict_Values(KNeighborsClassifier(n_neighbors = 5), X_train, Y_train)
print_Report(Y_test, knc_Y, 'Метод k-ближайших соседей', name_values)

Метод RandomForestClassifier
              precision    recall  f1-score   support

           A       1.00      1.00      1.00        93
           B       1.00      1.00      1.00       108
           C       1.00      1.00      1.00       172

    accuracy                           1.00       373
   macro avg       1.00      1.00      1.00       373
weighted avg       1.00      1.00      1.00       373

Метод Support Vector Machines
              precision    recall  f1-score   support

           A       1.00      0.98      0.99        93
           B       0.98      0.93      0.95       108
           C       0.96      1.00      0.98       172

    accuracy                           0.97       373
   macro avg       0.98      0.97      0.97       373
weighted avg       0.97      0.97      0.97       373

Метод k-ближайших соседей
              precision    recall  f1-score   support

           A       0.99      1.00      0.99        93
           B       1.00      0.99      1.00 